In [1]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /private/var/folders/ty/h3r8lvfs5sgdk7b7hcdvl86r0000gn/T/pip-req-build-g1oe1dfe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /private/var/folders/ty/h3r8lvfs5sgdk7b7hcdvl86r0000gn/T/pip-req-build-g1oe1dfe
  Resolved https://github.com/huggingface/accelerate to commit 16eb6d76bf987c7d8d877ce5152f2e29878eab37
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.28.0.dev0-py3-none-any.whl size=289777 sha256=f72e7cfd035cd88863065dd1df0ab44afae7fb20487fcafbae9227f571dd9e2b
  Stored in directory: /private/var/folders/ty/h3r8lvfs5sgdk7b7hcdvl86r0000gn/T/pip-ephem-wheel-cache-pxrog9jk/wheels/18/af/f7/facfc4ea8d2484e23fc8489825221fe5826625fad79301dd99
Successfully built accelerate


In [3]:
import os
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datasets

In [4]:
from transformers import AutoTokenizer

In [5]:
import evaluate

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [13]:
# Upload the dataset
#dataset = datasets.load_dataset("turingmachine/hupd-npe")
dataset = datasets.load_dataset("/Users/jeongwoopark/Desktop/Patent_Litigation/hupd-npe")

Generating train split:   0%|          | 0/49746 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['application_number', 'decision', 'title', 'abstract', 'claims', 'description', 'background', 'summary', 'cpc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'ipc_label', 'npe_litigated_count'],
        num_rows: 49746
    })
})

In [15]:
## TODO: SAMPLING METHOD
# Litigated to non-litigated ratio is 1:2.
# Year
# Year + CPC

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['application_number', 'decision', 'title', 'abstract', 'claims', 'description', 'background', 'summary', 'cpc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'ipc_label', 'npe_litigated_count'],
        num_rows: 49746
    })
})

In [17]:
# Create train and test splits
# Train set should contain all the data up to year 2015 (use filing_date)
# Test set should contain all the data from year 2016 onwards (use filing_date)
train_set = dataset.filter(lambda example: int(example["filing_date"][0:4]) <= 2014)
test_set = dataset.filter(lambda example: int(example["filing_date"][0:4]) > 2014)

Filter:   0%|          | 0/49746 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49746 [00:00<?, ? examples/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [19]:
SECTION = "abstract"

def tokenize_function(examples):
    return tokenizer(examples[SECTION], truncation=True)

In [20]:
train_set

DatasetDict({
    train: Dataset({
        features: ['application_number', 'decision', 'title', 'abstract', 'claims', 'description', 'background', 'summary', 'cpc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'ipc_label', 'npe_litigated_count'],
        num_rows: 43776
    })
})

In [21]:
train_set_tokenized = train_set.map(tokenize_function, batched=True)
test_set_tokenized = test_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/43776 [00:00<?, ? examples/s]

Map:   0%|          | 0/5970 [00:00<?, ? examples/s]

In [22]:
# Set the label 0 if npe_litigated_count = 0 and 1 otherwise
train_set_tokenized = train_set_tokenized.map(lambda example: {"label": 0 if example["npe_litigated_count"] == 0 else 1})
test_set_tokenized = test_set_tokenized.map(lambda example: {"label": 0 if example["npe_litigated_count"] == 0 else 1})

Map:   0%|          | 0/43776 [00:00<?, ? examples/s]

Map:   0%|          | 0/5970 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
# Counter
from collections import Counter

Counter(test_set_tokenized["train"]["label"])

Counter({0: 3980, 1: 1990})

In [34]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [35]:
id2label = {0: "NOT_LITIGATED", 1: "LITIGATED"}
label2id = {"NOT_LITIGATED": 0, "LITIGATED": 1}

In [36]:
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
train_set_tokenized

DatasetDict({
    train: Dataset({
        features: ['application_number', 'decision', 'title', 'abstract', 'claims', 'description', 'background', 'summary', 'cpc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'ipc_label', 'npe_litigated_count', 'input_ids', 'attention_mask', 'label'],
        num_rows: 43776
    })
})

In [38]:
training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set_tokenized["train"],
    eval_dataset=test_set_tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/16416 [00:00<?, ?it/s]

KeyboardInterrupt: 